In [1]:
from Model3 import *
from Model3_config import *
from Model3_MT5 import *

In [2]:
MODEL_PATH = "D:/Programing/AI Trader/Model/cnnLstmModelv.1.0/Model v.1.0_Train-loss 0.2628_Val-loss 0.9739_Val-Accuracy 0.7121_at 20241227-175258.model"

model.load_state_dict(torch.load(MODEL_PATH, weights_only=False))

model.eval()

print(f"{MODEL_PATH.split('/')[-1]} is loaded.")

Model v.1.0_Train-loss 0.2628_Val-loss 0.9739_Val-Accuracy 0.7121_at 20241227-175258.model is loaded.


In [ ]:
# Initialize a deque to store the last three predictions
recent_predictions = deque(maxlen=VOTE_LEN)

while True:
    try:
        # Fetch real-time data
        real_time_df = fetch_realtime_data(f"{time_frame}min")
        
        # Get predicted signal
        predicted_signal = predict(model, real_time_df, seq_length, scaler, device)
        prediction = conditions_map[predicted_signal]
        
        # Add the prediction to the deque
        recent_predictions.append(prediction)
        open_position(prediction)
        
        # Determine majority vote if at least 3 predictions are available
        if len(recent_predictions) == VOTE_LEN:
            prediction_counts = {pred: recent_predictions.count(pred) for pred in set(recent_predictions)}
            majority_vote = max(prediction_counts, key=prediction_counts.get)
            
            # open_position(majority_vote)
        else:
            # majority_vote = "Insufficient Data (No Trade!)"
            majority_vote = "Insufficient Data"
        
        # Print the real-time prediction and majority vote
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"Real-time Prediction -> {prediction}\t|\t{timestamp}")
        print(f"Majority Vote (Last 3): {majority_vote}\n")
    
    except ValueError as e:
        print(e)
    
    # Wait for the next interval
    time.sleep(time_frame * 60)

Trade executed successfully: sell 0.2 lots of EURUSD
Real-time Prediction -> Sell	|	2025-01-02 19:30:52
Majority Vote (Last 3): Insufficient Data

Trade executed successfully: buy 0.2 lots of EURUSD
Real-time Prediction -> Buy	|	2025-01-02 19:45:53
Majority Vote (Last 3): Insufficient Data

No clear majority vote. No trade executed.
Real-time Prediction -> Nothing	|	2025-01-02 20:00:56
Majority Vote (Last 3): Nothing

No clear majority vote. No trade executed.
Real-time Prediction -> Nothing	|	2025-01-02 20:15:59
Majority Vote (Last 3): Nothing

Trade executed successfully: buy 0.2 lots of EURUSD
Real-time Prediction -> Buy	|	2025-01-02 20:31:01
Majority Vote (Last 3): Nothing

